## Deep Dictionary Learning implementation

Julia implementation of the paper avaliable at: 

https://arxiv.org/pdf/1602.00203.pdf

In [ ]:
using Pkg
Pkg.add("PyCall")
Pkg.add("DataStructures")
Pkg.add("SparseArrays")
Pkg.add("ProgressBars")
Pkg.add("DelimitedFiles")
Pkg.add("Plots")
Pkg.add("Images")
using DataStructures
using SparseArrays
using LinearAlgebra
using ProgressBars
using PyCall
using Plots

In [ ]:
function init_dictionary(n::Int, K::Int)
    """
    Initialize the dictionary.
    Args: 
        n: dimension of input signal
        k = number of atoms in the dictionary
    """
    # D must be a full-rank matrix
    D = rand(n, K)
    while rank(D) != min(n, K)
        D = rand(n, K)
    end

    @inbounds for k in 1:K
        D[:, k] ./= norm(@view(D[:, k]))
    end
    return D
end

In [ ]:
function dense_dict_learning(x,nb_iters,n_atoms)
    """
    Computes a dense dictionary learned layer with the correspondent learned representation.
    
    Args:
    x - features to learn
    nb_iters: number of iterations for the algorithm
    n_atoms - number of atoms in the dictionary
    
    returns: 
    Dictionary (D) and dense coefficients (Z)
    
    """
    
    D = init_dictionary(size(x,1),n_atoms)
    Z = zeros(n_atoms,size(x,2))
    
    for i in ProgressBar(1:nb_iters)
        Z = inv((transpose(D)*D))*transpose(D)*x
        D = x*transpose(Z)*inv(Z*transpose(Z))
    end
    return D,Z
    
end

In [ ]:
function sparse_dict_learning(x,nb_iters,n_atoms,λ,α)
    """
    Computes a sparse dictionary learned layer with the correspondent learned representation,
    trought the use of a regularizing term with respect to the l1_norm.
    
    Args:
    x - features to learn
    nb_iters: number of iterations for the algorithm
    n_atoms - number of atoms in the dictionary
    
    returns: 
    Dictionary (D) and sparse coefficients (Z)
    
    """
    D = init_dictionary(size(x,1),n_atoms)
    Z = zeros(n_atoms,size(x,2))

    for i in ProgressBar(1:nb_iters)
        B = Z + (1/α)*transpose(D)*(x-D*Z)
        Ub, Sb, Vb = svd(transpose(B)*B, full=true)
        Z = B*inv(Vb)*max(0,norm(B)-λ/(2*α))
        D = x*transpose(Z)*inv(Z*transpose(Z))
    end
    return D,Z  
end

Learning the sucessive dictionaries: _(Deep)_

In [ ]:
function learn(X)
    λ = 0.01
    α = 0.01
    D1,Z1 = dense_dict_learning(X,nb_iters,300)
    D2,Z2 = dense_dict_learning(Z1,nb_iters,15)
    D3,Z3 = sparse_dict_learning(x,nb_iters,n_atoms,λ,α)
    
    return D1,D2,D3
end